In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import statsmodels.api as sm
import scipy.stats as stats
import re


from scipy.stats import chi2_contingency
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import precision_recall_fscore_support, auc, classification_report, fbeta_score, make_scorer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier, LinearRegression, Ridge, Lasso
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, make_scorer, mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer


from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

import kagglehub
import tqdm as notebook_tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\beert\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
final_model = pd.read_csv("data_preprocessed.csv")

In [14]:
final_model.describe()

,Rating,min_salary,max_salary,avg_salary,Company Age,JobTitle_numeric,contains_requirements,contains_team,contains_reporting,contains_information,contains_management,contains_skills,Company Offers,num_competitors,Location_num,Headquarters_num,Sector_num,Easy_Apply
count,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000
mean,3.725582,54267.199290,89984.021305,72125.610297,39.247670,7.648913,0.595206,0.734576,0.500666,0.526853,0.634709,0.786951,4.193076,0.608966,6.241012,72125.610297,6.837550,0.035508
std,0.634448,19571.034098,29310.015510,23595.818151,42.024827,3.078119,0.490961,0.441657,0.500111,0.499389,0.481618,0.409553,9.478334,1.148112,4.440955,2234.925395,4.131957,0.185102
min,1.000000,24000.000000,38000.000000,33500.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,52375.000000,0.000000,0.000000
25%,3.400000,41000.000000,70000.000000,58000.000000,17.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,72309.546314,3.000000,0.000000
50%,3.700000,50000.000000,87000.000000,69000.000000,24.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,7.000000,72309.546314,7.000000,0.000000
75%,4.100000,64000.000000,104000.000000,80500.000000,40.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,0.000000,11.000000,72309.546314,10.000000,0.000000
max,5.000000,113000.000000,190000.000000,150000.000000,343.000000,16.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,58.000000,4.000000,14.000000,150000.000000,19.000000,1.000000


## 3. MÈTRIQUES DE SELECCIÓ

Primer cal dividir les dades en conjunt d’entrenament i test (tipus 80%-20% o 70%-30%):

In [20]:
X = final_model.drop(columns=['avg_salary', 'min_salary', 'max_salary'])
y = final_model['avg_salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
y.mean(),y_test.mean(),y_train.mean()

(72125.61029738127, 71813.74722838137, 72203.66259711432)

**Normalització de dades**


Normalitzem les dades una vegada fet el split del conjunt d'entrenament i de test per minimitzar la pèrdua d'informació.

In [22]:
# Definir les columnes numèriques contínues a normalitzar
num_cols = ['Rating', 'Company Age',
            'Company Offers', 'num_competitors', 'Headquarters_num']  # excludes codificades tipus "JobTitle_numeric"

scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])

# Comprovació
X_train.head()

,Rating,Company Age,JobTitle_numeric,contains_requirements,contains_team,contains_reporting,contains_information,contains_management,contains_skills,Company Offers,num_competitors,Location_num,Headquarters_num,Sector_num,Easy_Apply
121,-0.370786,3.317633,6,1,1,1,1,1,1,-0.336997,-0.516865,9,0.077222,11,0
173,-0.211302,2.916835,15,1,1,1,1,1,1,-0.230084,2.144251,9,-1.531134,7,0
1245,-0.689753,-0.218821,6,1,0,1,1,1,0,-0.230084,-0.516865,11,0.077222,10,0
1323,-0.051818,0.677081,9,1,1,0,1,1,1,-0.336997,-0.516865,11,0.077222,3,0
999,0.905085,-0.360279,6,0,0,0,0,0,0,-0.336997,-0.516865,0,0.077222,3,0


FALTA PROVAR KNN!!!!

In [23]:
# Definir models a provar
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=0.1, random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Configurar cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Funcions de scoring
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score)

# Crear DataFrame per resultats
results = pd.DataFrame(columns=['Model', 'MAE_mean', 'MAE_std', 'R2_mean', 'R2_std'])

# Iterar sobre models
for name, model in models.items():
    mae_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=mae_scorer)
    r2_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=r2_scorer)
    
    results = pd.concat([results, pd.DataFrame({
        'Model': [name],
        'MAE_mean': [-mae_scores.mean()],  # Negatiu perquè sklearn retorna negatiu
        'MAE_std': [mae_scores.std()],
        'R2_mean': [r2_scores.mean()],
        'R2_std': [r2_scores.std()]
    })], ignore_index=True)

# Ordenar pel MAE més baix
results = results.sort_values(by='MAE_mean')
print(results)

C:\Users\beert\AppData\Local\Temp\ipykernel_15116\3483682961.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame({


              Model      MAE_mean     MAE_std   R2_mean    R2_std
4  GradientBoosting  14953.978492  717.179983  0.270982  0.020747
3      RandomForest  15612.843939  658.812029  0.188139  0.035818
1             Ridge  16098.911035  826.856891  0.187249  0.017695
2             Lasso  16099.578100  827.411351  0.187219  0.017724
0  LinearRegression  16099.611060  827.441192  0.187218  0.017725
